In [30]:
import numpy as np
import win32gui, win32ui, win32con
import numpy as np
import os
from time import time
import cv2 as cv
import pyautogui
class WindowCapture:

    # properties
    w = 0
    h = 0
    hwnd = None
    cropped_x = 0
    cropped_y = 0
    offset_x = 0
    offset_y = 0

    # constructor
    def __init__(self, window_name):
        # find the handle for the window we want to capture
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        # get the window size
        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        # account for the window border and titlebar and cut them off
        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

        # set the cropped coordinates offset so we can translate screenshot
        # images into actual screen positions
        self.offset_x = window_rect[0] + self.cropped_x
        self.offset_y = window_rect[1] + self.cropped_y

    def get_screenshot(self):

        # get the window image data
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        # convert the raw data into a format opencv can read
        #dataBitMap.SaveBitmapFile(cDC, 'debug.bmp')
        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        # free resources
        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        # drop the alpha channel, or cv.matchTemplate() will throw an error like:
        #   error: (-215:Assertion failed) (depth == CV_8U || depth == CV_32F) && type == _templ.type() 
        #   && _img.dims() <= 2 in function 'cv::matchTemplate'
        img = img[...,:3]

        # make image C_CONTIGUOUS to avoid errors that look like:
        #   File ... in draw_rectangles
        #   TypeError: an integer is required (got type tuple)
        img = np.ascontiguousarray(img)

        return img
    
    #static function allows it to be called without calling self
    @staticmethod
    def list_window_names():
        def winEnumHandler(hwnd, ctx):
            if win32gui.IsWindowVisible(hwnd):
                print(hex(hwnd), win32gui.GetWindowText(hwnd)+".")
        win32gui.EnumWindows(winEnumHandler, None)

    # translate a pixel position on a screenshot image to a pixel position on the screen.
    # pos = (x, y)
    # WARNING: if you move the window being captured after execution is started, this will
    # return incorrect coordinates, because the window position is only calculated in
    def get_screen_position(self, pos):
        return (pos[0] + self.offset_x, pos[1] + self.offset_y)

In [34]:
WindowCapture.list_window_names()

0x20910252 .
0x3001a .
0x4e068e .
0x40762 windowcapture - Jupyter Notebook - Google Chrome.
0x60748 New Tab - Google Chrome.
0x921fa VALORANT  .
-0x59dcf2a6 general - Discord.
0x9209e Task Manager.
0x34d053c Steam.
0xd11098c Data science goals - OneNote.
0x18b40ba0 ScreenCapTest.
0x9c07cc ScreenCapTest Properties.
0x7404a4 Anaconda Prompt (anaconda3).
0x260c44 Jupyter Notebook (anaconda3).
0xb970d4a GeForce Experience.
0x6006a Razer Synapse.
0x30242 Microsoft Store.
0x8d1fcc Microsoft Store.
0xa0506c0 Settings.
0x8a70d20 Movies & TV.
0x3ebc0d90 Movies & TV.
0x36710b4e GitHub Desktop.
0x8b206a2 Settings.
0x6a60c0e .
0x1aae08ae .
0x3046a Microsoft Text Input Application.
0x101b8 .
0x11d0a38 .
0x6210820 .
0x101e0 .
0x101b2 .
0x101b0 .
0x558069a NVIDIA GeForce Overlay.
0x4e00b2e .
0x3e03d4 .
0x20186 Program Manager.


In [35]:
# import cv2 as cv

# from windowcapture import WindowCapture

# Change the working directory to the folder this script is in.
# Doing this because I'll be putting the files from each video in their own folder on GitHub
os.chdir(os.path.dirname(os.path.abspath(r'C:\Users\notal\Documents\GitHub\Guilty-Gear-Segmentation\Media\ScreenCapTest')))


# initialize the WindowCapture class
wincap = WindowCapture('VALORANT  ')

loop_time = time()
while(True):

    # get an updated image of the game
    screenshot = wincap.get_screenshot()

    cv.imshow('GG-Val-Input', screenshot)

    # debug the loop rate
    print('FPS {}'.format(1 / (time() - loop_time)))
    loop_time = time()

    # press 'q' with the output window focused to exit.
    # waits 1 ms every loop to process key presses
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break

print('Done.')


<ipython-input-30-1a45232e63db>:58: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


FPS 11.49165312737166
FPS 49.99110856843184
FPS 41.65685738973254
FPS 76.8722554158572
FPS 50.00362422508345
FPS 29.40399876615911
FPS 35.707144316556565
FPS 30.29625188705821
FPS 37.02991136066674
FPS 66.64819169897667
FPS 55.51913378426675
FPS 58.82696812016999
FPS 76.90749399490254
FPS 83.31123249577912
FPS 49.98932112890923
FPS 83.25335450575625
FPS 52.645303811926546
FPS 62.48404493042934
FPS 58.765975929272976
FPS 76.95970642201834
FPS 71.3984849774449
FPS 49.989916927881005
FPS 71.4094252247344
FPS 52.56481144962591
FPS 83.19225658012178
FPS 50.0704803743673
FPS 71.41428863311313
FPS 58.800577588986556
FPS 52.62812903873421
FPS 83.31785224767088
FPS 90.25831719388853
FPS 91.5287288597927
FPS 76.90185365138153
FPS 90.77990606670562
FPS 62.536215893842254
FPS 83.31288733513428
FPS 47.61060661096985
FPS 83.31123249577912
FPS 71.41428863311313
FPS 52.618225612203936
FPS 55.52060361374016
FPS 90.92553491296148
FPS 55.542660398596304
FPS 49.988725344139205
FPS 71.41185684611979
FPS 49

FPS 58.80882208606161
FPS 76.93994203323916
FPS 90.88416034669555
FPS 76.76954333302828
FPS 62.56793364759234
FPS 43.444481272787534
FPS 83.40400485195569
FPS 71.41064101472716
FPS 71.38876312698926
FPS 44.31102096010818
FPS 83.23848458989065
FPS 50.016742588661785
FPS 76.90044369476733
FPS 83.31785224767088
FPS 52.619545853719735
FPS 94.12710951526033
FPS 40.10425969307262
FPS 99.87151463199753
FPS 66.70012563013852
FPS 55.459671021314854
FPS 58.90380024155268
FPS 83.30957772216263
FPS 62.4896305125149
FPS 83.30957772216263
FPS 76.9032636596993
FPS 52.608325912175296
FPS 66.66302171079818
FPS 83.32778384821695
FPS 66.63125119145961
FPS 55.554431184519004
FPS 41.65727112011601
FPS 71.383903194513
FPS 52.635393921140476
FPS 66.50974422402994
FPS 77.09834197272158
FPS 71.41428863311313
FPS 83.31454224023202
FPS 47.60844494892168
FPS 71.4130727189144
FPS 76.9032636596993
FPS 76.86802895629066
FPS 76.93429692945449
FPS 49.99408791837513
FPS 62.48404493042934
FPS 52.619545853719735
FPS 66.6

FPS 76.88493758363427
FPS 76.93147468818782
FPS 76.81734766762513
FPS 71.48366425223689
FPS 76.90608383145697
FPS 90.89597780859917
FPS 90.79759276096463
FPS 76.9695924247151
FPS 76.8666202397097
FPS 76.95547034108215
FPS 83.31454224023202
FPS 83.32281775199651
FPS 90.88416034669555
FPS 76.9046737197235
FPS 90.87234595718867
FPS 45.44995882276451
FPS 76.89903378985387
FPS 58.81129588603158
FPS 62.3169405402193
FPS 58.96425006677632
FPS 58.80469954855172
FPS 71.40091585380385
FPS 58.78656724785558
FPS 90.94130656317078
FPS 90.78973115719295
FPS 90.99062825407844
FPS 99.96672784040804
FPS 52.620205999322536
FPS 62.487768540865886
FPS 76.9032636596993
FPS 90.88416034669555
FPS 90.89597780859917
FPS 38.4526893845632
FPS 99.97625914714085
FPS 90.89597780859917
FPS 38.451279324538646
FPS 62.48311409715912
FPS 83.31288733513428
FPS 49.989916927881005
FPS 99.97387615006912
FPS 83.2682297353636
FPS 90.95116662329777
FPS 62.474737845567205
FPS 71.42158498790995
FPS 90.90779834409814
FPS 55.54118

FPS 58.78986319802647
FPS 31.248539754440337
FPS 83.31454224023202
FPS 90.88809915922684
FPS 39.991456903127386
FPS 71.19971481437472
FPS 91.23605673018359
FPS 90.88809915922684
FPS 99.97864225781846
FPS 62.362341466315776
FPS 90.78383584770893
FPS 55.67906544537369
FPS 66.65348737425906
FPS 55.538983050847456
FPS 76.9117247955404
FPS 90.88809915922684
FPS 45.44700400910175
FPS 62.48032176374199
FPS 71.42036882524222
FPS 83.1147748890298
FPS 91.10723982883333
FPS 38.45480466851867
FPS 58.81294520163778
FPS 83.30957772216263
FPS 49.99051274105504
FPS 71.4130727189144
FPS 71.39483897324165
FPS 52.62812903873421
FPS 66.65242817187897
FPS 71.40699376894004
FPS 83.32281775199651
FPS 83.30957772216263
FPS 90.88612971028624
FPS 76.90749399490254
FPS 66.65454661030417
FPS 58.81129588603158
FPS 99.95958055290752
FPS 41.66099506342064
FPS 90.7229624502509
FPS 50.03523924272609
FPS 62.48869951282013
FPS 62.48497579143389
FPS 83.27980303391311
FPS 49.963120026683185
FPS 83.26161786600497
FPS 52.68

...

FPS 83.47870392484674
FPS 90.88809915922684
FPS 43.468344197904464
FPS 90.89006869352288
FPS 83.31454224023202
FPS 99.96911049671084
FPS 90.89006869352288
FPS 34.474770472534786
FPS 76.90890421006307
FPS 90.89203831317991
FPS 83.31454224023202
FPS 99.9571983508496
FPS 43.470146237316946
FPS 30.296689564507624
FPS 99.97625914714085
FPS 83.21866629630364
FPS 62.53994572510661
FPS 83.24178855657213
FPS 52.65059061295708
FPS 99.97625914714085
FPS 99.89530092647725
FPS 100.05496183206107
FPS 90.88219106844922
FPS 99.97864225781846
FPS 38.45339445335778
FPS 52.618225612203936
FPS 83.31619721107624
FPS 45.44503434676144
FPS 32.24998462200898
FPS 29.40564794301579
FPS 30.291438269598817
FPS 50.00064373845145
FPS 71.41672058573131
FPS 62.48311409715912
FPS 55.54560262743176
FPS 66.65030986810743
FPS 52.62152634022106
FPS 90.79366178889947
FPS 83.35759286125962
FPS 71.3984849774449
FPS 83.36753393890004
FPS 55.54118940106201
FPS 83.31950735001986
FPS 39.98955046002765
FPS 83.3211625181271
FPS 58

FPS 58.81706889540183
FPS 62.39202677575307
FPS 62.580069528370856
FPS 76.79625018309653
FPS 50.034642362933624
FPS 31.241789755163758
FPS 29.40647260081889
FPS 28.565131816417292
FPS 90.86643991420958
FPS 83.3211625181271
FPS 34.47023726361985
FPS 83.33771781676568
FPS 62.485906680174004
FPS 43.46564141890421
FPS 47.61006617705484
FPS 99.97387615006912
FPS 37.028930617722104
FPS 29.394931599013233
FPS 90.9275060700659
FPS 83.36587692796947
FPS 90.87825276796741
FPS 47.60952575540875
FPS 71.41428863311313
FPS 62.48497579143389
FPS 66.6206677467518
FPS 58.832744206923635
FPS 55.54560262743176
FPS 76.90044369476733
FPS 52.58985643533321
FPS 33.33018650519306
FPS 83.30792301428089
FPS 90.74062696060403
FPS 77.02896181888303
FPS 76.848311621686
FPS 50.02390096128617
FPS 33.32144843255954
FPS 90.8979476843725
FPS 90.90779834409814
FPS 62.48311409715912
FPS 62.493354788724
FPS 66.65030986810743
FPS 62.48218329162198
FPS 58.81376989413167
FPS 58.80882208606161
FPS 47.61006617705484
FPS 66.643

FPS 62.454272015247625
FPS 66.65030986810743
FPS 71.35718538933973
FPS 62.52968975953009
FPS 71.4130727189144
FPS 66.65454661030417
FPS 71.4130727189144
FPS 66.57414050347607
FPS 71.48488257149674
FPS 66.63972036860503
FPS 76.94135343862932
FPS 66.65348737425906
FPS 76.81312724342538
FPS 71.49463062080251
FPS 66.65242817187897
FPS 71.3462611417296
FPS 76.98230673225167
FPS 71.40334689569465
FPS 71.4228011919966
FPS 71.41064101472716
FPS 71.41672058573131
FPS 76.82438273866217
FPS 71.37904392369089
FPS 71.51291538081192
FPS 76.90608383145697
FPS 71.41428863311313
FPS 76.87507331378299
FPS 71.4434830006132
FPS 66.59739599872975
FPS 62.525961151443774
FPS 66.6598432955611
FPS 66.62278416671961
FPS 58.83522001991892
FPS 62.485906680174004
FPS 58.806348494195504
FPS 62.4896305125149
FPS 58.80882208606161
FPS 62.46078241574958
FPS 58.830268602286274
FPS 62.48404493042934
FPS 62.48869951282013
FPS 62.48683759665092
FPS 58.80882208606161
FPS 62.48497579143389
FPS 66.65348737425906
FPS 71.39119

FPS 62.48869951282013
FPS 62.48683759665092
FPS 58.80882208606161
FPS 66.64819169897667
FPS 62.4896305125149
FPS 62.48497579143389
FPS 66.64819169897667
FPS 71.36811298281437
FPS 62.52502906890075
FPS 66.65030986810743
FPS 71.4082094761394
FPS 71.41672058573131
FPS 71.33898016804436
FPS 62.53994572510661
FPS 71.4130727189144
FPS 71.41428863311313
FPS 76.85816902440813
FPS 71.38633307803592
FPS 76.98230673225167
FPS 76.83423400318746
FPS 71.47513718005521
FPS 76.90749399490254
FPS 71.4094252247344
FPS 71.41428863311313
FPS 76.90044369476733
FPS 71.41550458871805
FPS 71.41428863311313
FPS 76.90185365138153
FPS 66.62384242713048
FPS 76.9032636596993
FPS 71.34747478183952
FPS 66.64925076671275
FPS 66.63442688061005
FPS 71.43618217120277
FPS 66.64925076671275
FPS 71.33898016804436
FPS 66.70967331488373
FPS 71.37539990470356
FPS 71.45808913724956
FPS 71.40577810313421
FPS 66.65136900316229
FPS 71.41793662415502
FPS 71.4130727189144
FPS 71.4130727189144
FPS 76.90044369476733
FPS 76.9074939949

error: (1400, 'GetWindowDC', 'Invalid window handle.')

# Steps:

1. Image Segmentation model
    - Create pixel map labels of sample images
    - Exploratory analysis with unsupervised clustering of images to see what algorithm is picking up
    - Optimize reduction of features and kernels/layers to create output image to be used in classification
2. Reinforcement Learning of Guilty Gear